**HEART ATTACK PREDICTION**

In [1]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import seaborn as sns
import pandas as pd
import joblib
import numpy as np

In [2]:
df= pd.read_csv('heart.csv')
features = df[['age', 'sex', 'trestbps', 'chol', 'thal', 'cp', 'exang']]
x = df[features.columns]
y = df['target']
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [3]:
scalar = StandardScaler()
x_scaled = scalar.fit_transform(x)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state= 42)

In [5]:
param_grid = {
    'n_estimators': [100, 150],
    'learning_rate':[0.05,0.1],
    'max_depth':[3,4]
}
grid =GridSearchCV(GradientBoostingClassifier(random_state = 42), param_grid ,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.05, 0.1], 'max_depth': [3, 4],
                         'n_estimators': [100, 150]},
             scoring='accuracy')

In [6]:
# best model evaluation and indha process la it has good accuracy

best_model = grid.best_estimator_
y_pred = best_model.predict(x_test)
acc = accuracy_score(y_test,y_pred) * 100
roc = roc_auc_score(y_test,y_pred)
print("Gradient Boosting with Top 7 Features + GridSearchCV")
print("Best Parameters:", grid.best_params_)
print(f"Accuracy: {acc:.2f}%")
print(f"AUC Score: {roc:.4f}")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

Gradient Boosting with Top 7 Features + GridSearchCV
Best Parameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 150}
Accuracy: 98.54%
AUC Score: 0.9854

Classification Report:

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       102
           1       1.00      0.97      0.99       103

    accuracy                           0.99       205
   macro avg       0.99      0.99      0.99       205
weighted avg       0.99      0.99      0.99       205



In [1]:
joblib.dump(best_model,"teacher_model")
soft_labels = best_model.predict_proba(x_train)
np.save("soft_label",soft_labels)


NameError: name 'joblib' is not defined

In [8]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(best_model, f)

with open("scaler.pkl", "wb") as f:
    pickle.dump(scalar, f)


In [9]:
import numpy as np 
import torch
import torch.nn as nn  
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

In [10]:
# training phase da mapla
features = df[['age', 'sex', 'trestbps', 'chol', 'thal', 'cp', 'exang']]
x = df[features.columns]
y = df['target']

In [11]:
scalar = StandardScaler()
x_scaled= scalar.fit_transform(x)

In [12]:
soft_labels = np.load('soft_label.npy')

In [13]:
class StudentModel(nn.Module):
    def __init__(self, input_dim):
        super(StudentModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )

    def forward(self, x):
        return self.net(x)


studentmodel = StudentModel(x_train.shape[1])

x_tensor = torch.tensor(x_train.values, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.long)
soft_tensor = torch.tensor(soft_labels, dtype=torch.float32)
 


criterion_ce = nn.CrossEntropyLoss()
criterion_kl = nn.KLDivLoss(reduction="batchmean")
optimizer = optim.Adam(studentmodel.parameters(), lr=0.001)

epochs = 50
alpha = 0.5
losses = []

for epoch in range(epochs):
    studentmodel.train()
    optimizer.zero_grad()

    outputs = studentmodel(x_tensor) 
    softmax_outputs = torch.log_softmax(outputs, dim=1)  

    loss_soft = criterion_kl(softmax_outputs, soft_tensor)
    loss_hard = criterion_ce(outputs, y_tensor)
    loss = alpha * loss_hard + (1 - alpha) * loss_soft

    loss.backward()
    optimizer.step()

    losses.append(loss.item())

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")



Epoch 0, Loss: 6.8483
Epoch 10, Loss: 2.8921
Epoch 20, Loss: 1.0230
Epoch 30, Loss: 0.9255
Epoch 40, Loss: 0.8788


In [14]:
import torch
import numpy as np

custom_input = np.array([[35, 0, 110, 170, 2, 1, 0]])
custom_scaled = scalar.transform(custom_input)  
custom_tensor = torch.tensor(custom_scaled, dtype=torch.float32)


studentmodel.eval()
with torch.no_grad():
    output = studentmodel(custom_tensor)
    prediction = torch.argmax(output, dim=1).item()

if prediction == 1:
    print("❗ High Risk of Heart Attack — Consult a doctor immediately.")
else:
    print("✅ No Significant Risk of Heart Attack — Keep maintaining a healthy lifestyle.")

✅ No Significant Risk of Heart Attack — Keep maintaining a healthy lifestyle.


c:\Users\tharu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
